In [52]:
import pandas as pd

df = pd.read_csv("ScrapeDataFromTiktok.csv")
df.head()

,Username,Comment,Target
0,ANDY TORAJA,tapi sy heran knp tdk dari awal di persoalkan ...,0
1,Juan Felix Michael,jelas2.... pemungutan ulang dan 02 didiskualif...,0
2,818anakrantau,trus apa fungsi dkpp ketika ada pelanggaran tp...,1
3,ArayamirShop,"Yakin Pemilu ulang, KPU sdh jadwalkan bulan Ju...",0
4,Faizi,02 Nyungsep,0


In [53]:
print(df.iloc[3, 1])

Yakin Pemilu ulang, KPU sdh jadwalkan bulan Juni utk pemilu ulang


In [54]:
df.shape

(536, 3)

Check Duplicate Data

In [55]:
duplicates = df[df.duplicated()]
print("Baris yang duplikat:")
print(len(duplicates))

Baris yang duplikat:
0


Pra-Proses

Case Folding

In [56]:
df['Comment'] = df['Comment'].str.casefold()

In [57]:
df.head()

,Username,Comment,Target
0,ANDY TORAJA,tapi sy heran knp tdk dari awal di persoalkan ...,0
1,Juan Felix Michael,jelas2.... pemungutan ulang dan 02 didiskualif...,0
2,818anakrantau,trus apa fungsi dkpp ketika ada pelanggaran tp...,1
3,ArayamirShop,"yakin pemilu ulang, kpu sdh jadwalkan bulan ju...",0
4,Faizi,02 nyungsep,0


In [58]:
print(df.iloc[3, 1])

yakin pemilu ulang, kpu sdh jadwalkan bulan juni utk pemilu ulang


Tokenizing

In [59]:
from nltk.tokenize import RegexpTokenizer
tokenizer     = RegexpTokenizer(r'\w+')

# Tokenize the comments
df['Comment'] = df['Comment'].apply(lambda x: tokenizer.tokenize(x))

In [60]:
df.head()

,Username,Comment,Target
0,ANDY TORAJA,"[tapi, sy, heran, knp, tdk, dari, awal, di, pe...",0
1,Juan Felix Michael,"[jelas2, pemungutan, ulang, dan, 02, didiskual...",0
2,818anakrantau,"[trus, apa, fungsi, dkpp, ketika, ada, pelangg...",1
3,ArayamirShop,"[yakin, pemilu, ulang, kpu, sdh, jadwalkan, bu...",0
4,Faizi,"[02, nyungsep]",0


In [61]:
print(df.iloc[3, 1])

['yakin', 'pemilu', 'ulang', 'kpu', 'sdh', 'jadwalkan', 'bulan', 'juni', 'utk', 'pemilu', 'ulang']


Normalization

In [62]:
normalization_df = pd.read_csv("normalization_mappings.csv", encoding='ISO-8859-1')

normalization_dict = pd.Series(normalization_df['standard'].values, index=normalization_df['non_standard']).to_dict()

In [63]:
def normalize_tokens(tokens, norm_dict):
    return [norm_dict.get(token, token) for token in tokens]

df['Comment'] = df['Comment'].apply(lambda x: normalize_tokens(x, normalization_dict))

In [64]:
df.head()

,Username,Comment,Target
0,ANDY TORAJA,"[tapi, saya, heran, kenapa, tidak, dari, awal,...",0
1,Juan Felix Michael,"[jelas jelas, pemungutan, ulang, dan, 02, didi...",0
2,818anakrantau,"[terus, apa, fungsi, dkpp, ketika, ada, pelang...",1
3,ArayamirShop,"[yakin, pemilihan umum, ulang, komisi pemeliha...",0
4,Faizi,"[02, nyungsep]",0


In [65]:
print(df.iloc[3, 1])

['yakin', 'pemilihan umum', 'ulang', 'komisi pemelihan umum', 'sudah', 'jadwalkan', 'bulan', 'juni', 'untuk', 'pemilihan umum', 'ulang']


Remove Stopwords

In [66]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [67]:
stop_words

{'ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [68]:
def remove_stopwords(tokens, stop_words):
    return [token for token in tokens if token not in stop_words]

df['Comment'] = df['Comment'].apply(lambda x: remove_stopwords(x, stop_words))

In [69]:
df.head()

,Username,Comment,Target
0,ANDY TORAJA,"[heran, persoalkan, prabowo, gibran, lmbt, mnd...",0
1,Juan Felix Michael,"[jelas jelas, pemungutan, ulang, 02, didiskual...",0
2,818anakrantau,"[fungsi, dkpp, pelanggaran, sanksi, sebatas, t...",1
3,ArayamirShop,"[pemilihan umum, ulang, komisi pemelihan umum,...",0
4,Faizi,"[02, nyungsep]",0


In [70]:
print(df.iloc[3, 1])

['pemilihan umum', 'ulang', 'komisi pemelihan umum', 'jadwalkan', 'juni', 'pemilihan umum', 'ulang']


Stemming

In [71]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Function to stem tokens
def stem_tokens(tokens, stemmer):
    return [stemmer.stem(token) for token in tokens]

# Apply stemming
df['Comment'] = df['Comment'].apply(lambda x: stem_tokens(x, stemmer))

In [36]:
df.head()

,Username,Comment,Target
0,ANDY TORAJA,"[heran, soal, prabowo, gibran, lmbt, mndaftar,...",0
1,Juan Felix Michael,"[jelas jelas, mungut, ulang, 02, diskualifikasi]",0
2,818anakrantau,"[fungsi, dkpp, langgar, sanksi, batas, tegur]",1
3,ArayamirShop,"[pilih umum, ulang, komisi pemelihan umum, jad...",0
4,Faizi,"[02, nyungsep]",0


In [72]:
print(df.iloc[3, 1])


['pilih umum', 'ulang', 'komisi pemelihan umum', 'jadwal', 'juni', 'pilih umum', 'ulang']
